## This Jupyter notebook is targeted towards QC analysis of gene models without alleles in a phased assembly
The script will pick up were *_defining alleles left off. In general it can take a list of genes without alleles in a phased assembly and can look why those were missed

It will consider the following options:

#### The allele was left out of the annotation in the other haplotig:
* It takes the gene sequence of a single allele gene and blast its against the other haplotic
* If there is an gene sequence it pulls out a region around this and align the protein sequence using exonerate
* The exonerate alignment is used to scan for matches without frame shifts and without stop codons
* In cases where a good aligment is possible these alleles are written noted

#### The allele was left out because it was not phased in the first place
This step relies on the Pst_104E_v12_coverage_analysis_training script to give out homozygous regions when doing p mapping when compare to p to ph mapping. Might need to be adapted a bit more.
* All remaining single allele genes are tested if they fall into a homozygous coverage area
* If they do not overlap with an ortholgos contig alignement p on h mapping and reverse
* Maybe if they overlap with a unique coverage area. Only possible for p alleles so far. 

#### Else to consider would:
* look for gene that have no-haplotig aligned and their variation in terms of SNPs


##### script considerations

What to do when mulitple filtered allele files are present? Mabye have previous script write out different options to different folders if they already exist.

In [1]:
%matplotlib inline

In [177]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil
from Bio.Seq import Seq
import pysam
from Bio import SearchIO

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [17]:
def blast_outfmt6_to_bed(x):
    "Quick function that converts a blast outfmt6 file to a bed file."
    blast_fo = open(x, 'r')
    blast_lines = blast_fo.readlines()
    bed_file_name = x + '.bed'
    bed_fo = open(bed_file_name, 'w+')
    for l in blast_lines:
        content = l.split('\t')
        if int(content[8]) - int(content[9]) < 1:
            print(content[1], int(content[8]) -1, content[9], content[0], content[10], "+", sep="\t", file=bed_fo) 
        else:
            print(content[1], int(content[9]) -1, content[8],  content[0], content[10], "-", sep = "\t", file=bed_fo)
    blast_fo.close()
    bed_fo.close()
    return bed_file_name

In [68]:
def pwh_filter (x):
    p_contig = x.split('.')[2]
    if p_contig in pwh_set:
        return 1
    else:
        return 0

In [71]:
def same_contig_blast(x,y):
    '''Function that checks if the blast hit in columne x is on the same contig as the the query sequence in
    column y.
    '''
    q_contig = x.split('.')[2].split('_')[1]
    hit_contig = y.split('_')[1]
    if q_contig == hit_contig:
        return True
    else:
        return False

In [163]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
ALLELE_PATH =os.path.join(BASE_AA_PATH ,'allele_analysis/alleles')
BLAST_DB = os.path.join(BASE_AA_PATH, 'blast_DB')
OUT_PATH = os.path.join(BASE_AA_PATH, 'allele_analysis', 'no_alleles_QC')
OUT_PATH_tmp = os.path.join(OUT_PATH, 'tmp')
EXONERATE_PATH = os.path.join(OUT_PATH_tmp, 'exonerate')
if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH)
if not os.path.isdir(OUT_PATH_tmp):
    os.mkdir(OUT_PATH_tmp)
if not os.path.isdir(EXONERATE_PATH):
    os.mkdir(EXONERATE_PATH)

In [165]:
#clean up the tmp folder?
clean_up = False
exonerate_script_name = 'exonerate_alignments_vulgar.sh'

In [5]:
#genomes
p_genome = 'Pst_104E_v12_p_ctg'
h_genome = 'Pst_104E_v12_h_ctg'

In [6]:
#Define ENV parameters for blast hits and threads used in blast analysis
n_threads = 4
e_value = 1e-3
blast_stderr_dict ={} #keep track of all the blast outputs and errors if so
#propagate cut off from previous files from defining alleles
Qcov_cut_off = 80 #this defines the mimimum coverage of the Query to be required for filtering. Will become part of name.
PctID_cut_off = 70 #this defines the mimimum PctID accross the alignment to be required for filtering. Will become part of name.

In [8]:
protein_fa_files = [os.path.join(BASE_A_PATH, x) for x in os.listdir(BASE_A_PATH) if x.endswith('protein.fa')]

In [174]:
#read in protein ids for p and h contigs and store names in a list in a dict with unique key id [first part of
#file name].
fa_protein_dict = {}
fa_protein_seq_dict = {}
fa_protein_length_dict = {}
for file in protein_fa_files:
    seq_list = []
    length_list =[]
    for seq in SeqIO.parse(open(file), 'fasta'):
        fa_protein_seq_dict[seq.id] = seq
        fa_protein_length_dict[seq.id] = len(seq)


In [10]:
#get the file names of the no allele cases including the filtered settings with Qcov and PctID cut offs and the no alleles
#at all that in principle is all p and h proteins without a blast hit with a given e-value right now 0.001
filtered_no_alleles = [os.path.join(ALLELE_PATH, x) for x in os.listdir(ALLELE_PATH)\
                       if (x.split('.')[1] == 'no' and 'Qcov' in x and 'PctID' in x and x.startswith(p_genome) )or \
                           (x.startswith(h_genome) and 'Qcov' in x and 'PctID' in x and 'no.no_p_hits' in x)]
filtered_no_alleles_dict = {}
for x in filtered_no_alleles:
    key = x.split('/')[-1].split('.')[0]
    filtered_no_alleles_dict[key] = x
    
no_alleles_at_all = [os.path.join(ALLELE_PATH, x) for x in os.listdir(ALLELE_PATH)\
                       if (x.split('.')[1] == 'no' and 'Qcov' not in x and 'PctID' not in x and x.startswith(p_genome) )or \
                           (x.startswith(h_genome) and 'Qcov' not in x and 'PctID' not in x and 'no.no_p_hits' in x)]
no_alleles_at_all_dict ={}
for x in no_alleles_at_all:
    key = x.split('/')[-1].split('.')[0]
    no_alleles_at_all_dict[key] = x

###### Might want to be incooporated in the script in future
Pull gff and genome fasta files over into the tmp folder make gene gff and pull out gene sequences with bedtools getfasta on the command line using subproccesses. See below ideas from original script

In [11]:
#get the gene.fa files and put them in a dict that has the genome as a key
gene_fa_files = [os.path.join(BASE_A_PATH, x) for x in os.listdir(BASE_A_PATH) if x.endswith('gene.fa')]
gene_fa_files_dict = {}
for x in gene_fa_files:
    key = x.split('/')[-1].split('.')[0]
    gene_fa_files_dict[key] = x

In [12]:
#generate the blast databases if not already present
os.chdir(BLAST_DB)
blast_dir_content = os.listdir(BLAST_DB)
for x in blast_dir_content:
    if x.endswith('.fa') and ({os.path.isfile(x + e) for e in ['.psq', '.phr', '.pin'] } != {True}\
           and {os.path.isfile(x + e) for e in ['.nin', '.nhr', '.nsq'] } != {True} ):

        make_DB_options = ['-in']
        make_DB_options.append(x)
        make_DB_options.append('-dbtype')
        if 'protein' in x:
            make_DB_options.append('prot')
        else:
            make_DB_options.append('nucl')
        make_DB_command = 'makeblastdb %s' % ' '.join(make_DB_options)
        make_DB_stderr = subprocess.check_output(make_DB_command, shell=True, stderr=subprocess.STDOUT)
        print('%s is done!' % make_DB_command)
print("All databases generated and ready to go!")

All databases generated and ready to go!


In [13]:
#get the blast db files and put them in a dict that has the genome as a key
gene_blast_db = [os.path.join(BLAST_DB, x) for x in os.listdir(BLAST_DB) if x.endswith('gene.fa')]
gene_blast_db_dict ={}
for x in gene_blast_db:
    key = x.split('/')[-1].split('.')[0]
    gene_blast_db_dict[key] = x
genome_blast_db = [os.path.join(BLAST_DB, x) for x in os.listdir(BLAST_DB) if x.endswith('_ctg.fa')]
genome_blast_db_dict ={}
for x in genome_blast_db:
    key = x.split('/')[-1].split('.')[0]
    genome_blast_db_dict[key] = x


In [14]:
#using the dictionary approach to stich together all the different input files. The key is always the genome. In this cases
#being the part of the file name before the first '.'
if len(filtered_no_alleles) != 2:
    print("This script right now is only designed for one set of filter files.")
    print("Please hold!")
else:
    print("One pair of filtered non-allele files given. Good to go!")
    
#simply pulls in the gene sequences of missing alleles. Do this on the filtered set as the unfiltered set is a subset anyway
no_filtered_allele_gene_dict = {}
for no_alleles_key in filtered_no_alleles_dict.keys():
    #read in all the alleles from file this assumes that only one filter setting was run in the allele folder
    no_filtered_allele_list = pd.read_csv(os.path.join(ALLELE_PATH, filtered_no_alleles_dict[no_alleles_key]), header=None, sep='\t')[0].tolist()
    #convert from proteins ids to gene ideas
    no_filtered_allele_list =  [x.replace('evm.model', 'evm.TU') for x in no_filtered_allele_list]
    
    no_filtered_allele_seq = []
    for seq in SeqIO.parse(open(gene_blast_db_dict[no_alleles_key]), 'fasta'):
        if seq.id in no_filtered_allele_list:
            no_filtered_allele_seq.append(seq)
    #get the proper file name
    out_f_prefix = filtered_no_alleles_dict[no_alleles_key].split('/')[-1]
    out_f = out_f_prefix + '.gene.fa'
    f_handle = open(os.path.join(OUT_PATH, out_f),'w') #need to generate handle for writing and
    SeqIO.write(no_filtered_allele_seq, f_handle, 'fasta')
    f_handle.close() #closing file afterwards again
    no_filtered_allele_gene_dict[no_alleles_key] = os.path.join(OUT_PATH, out_f)

One pair of filtered non-allele files given. Good to go!


In [38]:
#do the gene against other haplotype blast
no_filtered_allele_gene_genome_blast_dict ={}
for no_alleles_key in no_filtered_allele_gene_dict.keys():
    blast_options = ['-query']
    query = no_filtered_allele_gene_dict[no_alleles_key]
    blast_options.append(query)
    blast_options.append('-db')
    if no_alleles_key == p_genome:
        db = genome_blast_db_dict[h_genome]
    elif no_alleles_key == h_genome:
        db = genome_blast_db_dict[p_genome]
    else:
        print("There is something wrong with the file name prefixes and the genome (h and p) provided!")
    blast_options.append(db)
    blast_options.append('-outfmt 6')
    blast_options.append('-evalue')
    blast_options.append(str(e_value))
    blast_options.append('-num_threads')
    blast_options.append(str(n_threads))
    #blast_options.append('-max_target_seqs 1')
    blast_options.append('>')
    if 'gene' in query:
        out_name_list = [ query.split('/')[-1], 'db_' + db.split('/')[-1], str(e_value), 'blastn.outfmt6']
        out_name = os.path.join(OUT_PATH_tmp ,'.'.join(out_name_list))
        blast_options.append(out_name)
        blast_command = 'blastn %s' % ' '.join(blast_options)
    no_filtered_allele_gene_genome_blast_dict[no_alleles_key] = out_name
    print(blast_command)
    if not os.path.exists(out_name):
        blast_stderr_dict[blast_command] = subprocess.check_output(blast_command, shell=True, stderr=subprocess.STDOUT)
        print("New blast run and done!")
    else:
        blast_stderr_dict[blast_command] = 'Previously done already!'
        print('Previously done already!')

blastn -query /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles_QC/Pst_104E_v12_h_ctg.no.no_p_hits.Qcov80.PctID70.alleles.gene.fa -db /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/blast_DB/Pst_104E_v12_p_ctg.fa -outfmt 6 -evalue 0.001 -num_threads 4 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles_QC/tmp/Pst_104E_v12_h_ctg.no.no_p_hits.Qcov80.PctID70.alleles.gene.fa.db_Pst_104E_v12_p_ctg.fa.0.001.blastn.outfmt6
Previously done already!
blastn -query /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles_QC/Pst_104E_v12_p_ctg.no.Qcov80.PctID70.alleles.gene.fa -db /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/blast_DB/Pst_104E_v12_h_ctg.fa -outfmt 6 -evalue 0.001 -num_threads 4 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles

In [39]:
no_filtered_allele_gene_genome_blast_dict.items()

dict_items([('Pst_104E_v12_h_ctg', '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles_QC/tmp/Pst_104E_v12_h_ctg.no.no_p_hits.Qcov80.PctID70.alleles.gene.fa.db_Pst_104E_v12_p_ctg.fa.0.001.blastn.outfmt6'), ('Pst_104E_v12_p_ctg', '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles_QC/tmp/Pst_104E_v12_p_ctg.no.Qcov80.PctID70.alleles.gene.fa.db_Pst_104E_v12_h_ctg.fa.0.001.blastn.outfmt6')])

In [40]:
#now convert all the gene level against genome blast hits to bed files
no_filtered_allele_gene_genome_blast_bed_dict = {}
for key, value in no_filtered_allele_gene_genome_blast_dict.items():
    no_filtered_allele_gene_genome_blast_bed_dict[key] = blast_outfmt6_to_bed(value)
    

In [203]:
#here track what happens with the no_alleles. Meaning how many of those have a gene vs. genome hit and how many don't 

#these dict will hold the list of SeqIO.records of blast hit regions for each no_allele hiting the other haplotig split into
#the id of the contig will be h/pcontig_xxx_start_end of DNA sequence

#hit on associated contig
no_filtered_allele_gene_genome_hit_asso_contig_dict = {}

#hit on unlinked contigs
no_filtered_allele_gene_genome_hit_no_asso_contig_dict = {}


for key, no_filtered_alllele_fn in filtered_no_alleles_dict.items():
    no_filtered_alleles = pd.read_csv(no_filtered_alllele_fn, sep='\t', header=None)[0].unique()
    genome_hits_header = ['Contig', 'start', 'end', 'blast_query', 'e-value', 'strand']
    gene_genome_hits_df = pd.read_csv(no_filtered_allele_gene_genome_blast_bed_dict[key], sep='\t', \
                                         names = genome_hits_header, header=None)
    gene_genome_hits_df['Protein_ID'] = gene_genome_hits_df['blast_query'].str.replace('evm.TU', 'evm.model')
    #get all alleles with no gene vs. genome hit and save them to file with ending 'no_allele_no_gene_genome_blast_hit.txt'
    no_filtered_allele_gene_no_genome_hit = np.setdiff1d(no_filtered_alleles, gene_genome_hits_df.Protein_ID.unique(), assume_unique= True)
    out_fn = os.path.join(OUT_PATH, key +'no_allele_no_gene_genome_blast_hit.txt')
    no_filtered_allele_gene_no_genome_hit_dict[key] = out_fn
    np.savetxt(out_fn, no_filtered_allele_gene_no_genome_hit, fmt='%s')
    #now filter out the best hit on an associated contig
    gene_genome_hits_df['asso_contig'] = gene_genome_hits_df['blast_query'].combine(gene_genome_hits_df['Contig'], func=same_contig_blast)
    tmp_same_contig_df = ''
    tmp_same_contig_df = gene_genome_hits_df[gene_genome_hits_df['asso_contig'] == True]
    #now filter out the best hit on an not-associated contig <- not for now as this might get a bit complicated with paraglogs and such
    tmp_diff_contig_df = ''
    tmp_diff_contig_df_grouped = gene_genome_hits_df[gene_genome_hits_df['asso_contig'] == False].groupby('blast_query')
    tmp_diff_contig_best_hits = tmp_diff_contig_df_grouped.apply(lambda g: g[g['e-value'] == g['e-value'].min()])
    #now get all query protein ids
    tmp_protein_id = gene_genome_hits_df['Protein_ID'].unique()
    genome_name = ''
    if key == p_genome:
        genome_name = os.path.join(BASE_A_PATH, h_genome+'.fa')
    elif key == h_genome:
        genome_name = os.path.join(BASE_A_PATH, p_genome+'.fa')
    genome_fa = pysam.FastaFile(genome_name)
    for protein_id in tmp_protein_id:
        #now loop through the protein_ids of no_alleles hiting the associated contig aka same contig
        #could do something like gene_genome_hits_df.pivot_table(columns=['Protein_ID', 'Contig'], aggfunc={'start' : 'min', 'end':'min'})
        tmp_protein_id_df = gene_genome_hits_df[(gene_genome_hits_df['Protein_ID'] == protein_id) & (gene_genome_hits_df['asso_contig'] == True) ]
        
        if len(tmp_protein_id_df) < 1:
            continue
        tmp_hit_contig = tmp_protein_id_df["Contig"].unique()
        tmp_gene_genome_seq_list = [] #saves SeqIO records from blast hits and suroundings
        #now loop through the associated contig hits incase we have multiple associated contigs hit
        for hit in tmp_hit_contig:
            tmp_df_2 = tmp_protein_id_df[tmp_protein_id_df['Contig'] == hit]
            #get the smallest starting point on the specific contig
            start = tmp_df_2['start'].min() - 30000
            if start < 1:
                start = 1
            end = tmp_df_2['end'].max() + 30000
            seq = genome_fa.fetch(hit, start, end)
            seq_r = '' #initialize empty SeqIO record
            seq_id = hit + '_' + str(start) + '_' + str(end)
            seq_ob = Seq(seq)
            seq_ob.alphabet = 'fasta'
            seq_r = SeqRecord(seq_ob)
            seq_r.id = seq_id
            tmp_gene_genome_seq_list.append(seq_r)
        no_filtered_allele_gene_genome_hit_asso_contig_dict[protein_id] = tmp_gene_genome_seq_list
        #now loop through the protein_ids of no_alleles hiting unassociated contig aka diff_contig
        tmp_protein_id_df = tmp_diff_contig_best_hits[(tmp_diff_contig_best_hits['Protein_ID'] == protein_id)]
        if len(tmp_protein_id_df) < 1:
            continue
        tmp_hit_contig = tmp_protein_id_df["Contig"].unique()
        tmp_gene_genome_seq_list = [] #saves SeqIO records from blast hits and suroundings
        #now loop through the associated contig hits incase we have multiple associated contigs hit
        #pull out the blast hit regions (for one contig start(min) and end(max) if mulitple hits on same contig.
        #save SeqIO.Records for each protein_id in a list
        for hit in tmp_hit_contig:
            tmp_df_2 = tmp_protein_id_df[tmp_protein_id_df['Contig'] == hit]
            #get the smallest starting point on the specific contig
            start = tmp_df_2['start'].min() - 30000
            if start < 1:
                start = 1
            end = tmp_df_2['end'].max() + 30000
            seq = genome_fa.fetch(hit, start, end)
            seq_r = '' #initialize empty SeqIO record
            seq_id = hit + '_' + str(start) + '_' + str(end)
            seq_ob = Seq(seq)
            seq_ob.alphabet = 'fasta'
            seq_r = SeqRecord(seq_ob)
            seq_r.id = seq_id
            tmp_gene_genome_seq_list.append(seq_r)
        no_filtered_allele_gene_genome_hit_no_asso_contig_dict[protein_id] = tmp_gene_genome_seq_list
    
    

In [204]:
#now write an exonerate script that aligns the protein sequences to the DNA sequences
EXONERATE_PATH_asso = os.path.join(EXONERATE_PATH, 'hit_associated_contigs')
EXONERATE_PATH_no_asso = os.path.join(EXONERATE_PATH, 'hit_nonassociated_contigs')
if not os.path.exists(EXONERATE_PATH_asso):
    os.mkdir(EXONERATE_PATH_asso)
if not os.path.exists(EXONERATE_PATH_no_asso):
    os.mkdir(EXONERATE_PATH_no_asso)
#open up the script
exonerate_script = os.path.join(OUT_PATH_tmp, exonerate_script_name)
out_exonerate = open(exonerate_script, 'w')
out_exonerate.write('#!/bin/bash\n')
for contig_key, contig_seq_list in no_filtered_allele_gene_genome_hit_asso_contig_dict.items():
    out_folder = os.path.join(EXONERATE_PATH_asso, contig_key)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_protein_fn = os.path.join(out_folder, contig_key + '.fa')
    out_handle = open(out_protein_fn, 'w')
    #write down the protein sequence
    SeqIO.write(fa_protein_seq_dict[contig_key], out_handle, 'fasta')
    out_handle.close()
    #write the exonerate script
    out_exonerate.write('cd %s\n'% out_folder)
    #write out all the genomic regions
    for seq in contig_seq_list:
        out_seq_name = os.path.join(out_folder, seq.id +'.fa')
        out_seq_handle = open(out_seq_name, 'w')
        SeqIO.write(seq, out_seq_handle, 'fasta')
        out_seq_handle.close()
        #write exonerate script the command
        out_exonerate.write('exonerate --model protein2genome --percent 20 -q %s -t %s --showalignment False -S > %s.vulgar_exn\n'\
                           %(out_protein_fn, out_seq_name, out_seq_name))

    out_exonerate.write('cd %s\n'% out_folder)
    
for contig_key, contig_seq_list in no_filtered_allele_gene_genome_hit_no_asso_contig_dict.items():
    out_folder = os.path.join(EXONERATE_PATH_no_asso, contig_key)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_protein_fn = os.path.join(out_folder, contig_key + '.fa')
    out_handle = open(out_protein_fn, 'w')
    #write down the protein sequence
    SeqIO.write(fa_protein_seq_dict[contig_key], out_handle, 'fasta')
    out_handle.close()
    #write the exonerate script
    out_exonerate.write('cd %s\n'% out_folder)
    #write out all the genomic regions
    for seq in contig_seq_list:
        out_seq_name = os.path.join(out_folder, seq.id +'.fa')
        out_seq_handle = open(out_seq_name, 'w')
        SeqIO.write(seq, out_seq_handle, 'fasta')
        out_seq_handle.close()
        #write exonerate script the command
        out_exonerate.write('exonerate --model protein2genome --percent 20 -q %s -t %s --showalignment False -S > %s.vulgar_exn\n'\
                           %(out_protein_fn, out_seq_name, out_seq_name))

    out_exonerate.write('cd %s\n'% out_folder)       

out_exonerate.close()

In [205]:
#now run the exonerate script
exonerate_command = 'bash %s' % exonerate_script
exonerate_stderr = subprocess.check_output(exonerate_command , shell=True, stderr=subprocess.STDOUT)

In [206]:
#no loop through the exonerate vulgar result and generate a dictionray of the results
#if hsps query range == (0, query_length) and not F in .vulgar_comp it is likely that the alignment is actually good
#and and the gene model might have been dropped for another reason
#a dict that has the protein ID as key and the results of exonerate as list as value for each contig [contig : True/False].
exonerate_best_hit_dict = {}
exonerate_no_filtered_allele_gene_genome_hit_asso_contig_dict = {}
exonerate_no_filtered_allele_asso_contig_bool_dict = {}
#now loop through the exonerate folders

for contig_key in no_filtered_allele_gene_genome_hit_asso_contig_dict.keys():
    out_folder = os.path.join(EXONERATE_PATH_asso, contig_key)
    query_length = fa_protein_length_dict[contig_key]
    
    exonerate_result_list = []
    counter = 0
    overall_best_score = 0
    overall_best_hit = ''
    #dummy value for best hit and score
    exonerate_best_hit_dict[contig_key] = ['dummy : 0']
    vulgar_exn_list = [os.path.join(out_folder, x) for x in os.listdir(out_folder) if x.endswith('vulgar_exn')]
    opt_query_range = (0, query_length)
    #loop through vulgar parser and see if hit is 
    for fname in vulgar_exn_list:
        best_score = 0
        best_hit = ''
        result = SearchIO.parse(fname, 'exonerate-vulgar')
        genome_region = fname.split('/')[-1].split('.')[0]
        for hit in result:
            #loop through all hsps hits
            for hsps in hit.hsps:
                hsps_range = hsps.query_range
                vulgar_list = hsps.vulgar_comp.strip(' ').split(' ')
                #print(hsps_range, vulgar_list)
                #this is the contition for something being a potential protein alignment that
                #has been left out
                if hsps_range == opt_query_range and 'F' not in vulgar_list:
                    counter += 1
                    if hsps.score > best_score:
                        best_hit = hsps.hit_id
                        best_score = hsps.score
                    if hsps.score > overall_best_score:
                        overall_best_hit = hsps.hit_id
                        overall_best_score = hsps.score
                    #print(key)
        if best_score > 0:
            exonerate_result_list.append('%s : True' % genome_region)
        else:
            exonerate_result_list.append('%s : False' % genome_region)
            
    exonerate_no_filtered_allele_gene_genome_hit_asso_contig_dict[contig_key] = exonerate_result_list
    
    
    
    if counter > 0:
        exonerate_no_filtered_allele_asso_contig_bool_dict[contig_key] = True
        
        if contig_key in exonerate_best_hit_dict.keys():
            if int(exonerate_best_hit_dict[contig_key][0].split(':')[1][1:]) > overall_best_score:
                exonerate_best_hit_dict[contig_key] = 'overall_best_hit : %s' % overall_best_score
    else:
        exonerate_no_filtered_allele_asso_contig_bool_dict[contig_key] = False
    


In [207]:
#no loop through the exonerate vulgar result and generate a dictionray of the results
#if hsps query range == (0, query_length) and not F in .vulgar_comp it is likely that the alignment is actually good
#and and the gene model might have been dropped for another reason
#a dict that has the protein ID as key and the results of exonerate as list as value for each contig [contig : True/False].
exonerate_best_hit_dict = {}
exonerate_no_filtered_allele_gene_genome_hit_no_asso_contig_dict = {}
exonerate_no_filtered_allele_no_asso_contig_bool_dict = {}
#now loop through the exonerate folders

for contig_key in no_filtered_allele_gene_genome_hit_no_asso_contig_dict.keys():
    out_folder = os.path.join(EXONERATE_PATH_no_asso, contig_key)
    query_length = fa_protein_length_dict[contig_key]
    
    exonerate_result_list = []
    counter = 0
    overall_best_score = 0
    overall_best_hit = ''
    #dummy value for best hit and score
    exonerate_best_hit_dict[contig_key] = ['dummy : 0']
    vulgar_exn_list = [os.path.join(out_folder, x) for x in os.listdir(out_folder) if x.endswith('vulgar_exn')]
    opt_query_range = (0, query_length)
    #loop through vulgar parser and see if hit is 
    for fname in vulgar_exn_list:
        best_score = 0
        best_hit = ''
        result = SearchIO.parse(fname, 'exonerate-vulgar')
        genome_region = fname.split('/')[-1].split('.')[0]
        for hit in result:
            #loop through all hsps hits
            for hsps in hit.hsps:
                hsps_range = hsps.query_range
                vulgar_list = hsps.vulgar_comp.strip(' ').split(' ')
                #print(hsps_range, vulgar_list)
                #this is the contition for something being a potential protein alignment that
                #has been left out
                if hsps_range == opt_query_range and 'F' not in vulgar_list:
                    counter += 1
                    if hsps.score > best_score:
                        best_hit = hsps.hit_id
                        best_score = hsps.score
                    if hsps.score > overall_best_score:
                        overall_best_hit = hsps.hit_id
                        overall_best_score = hsps.score
                    #print(key)
        if best_score > 0:
            exonerate_result_list.append('%s : True' % genome_region)
        else:
            exonerate_result_list.append('%s : False' % genome_region)
            
    exonerate_no_filtered_allele_gene_genome_hit_no_asso_contig_dict[contig_key] = exonerate_result_list
    
    
    
    if counter > 0:
        exonerate_no_filtered_allele_no_asso_contig_bool_dict[contig_key] = True
        
        if contig_key in exonerate_best_hit_dict.keys():
            if int(exonerate_best_hit_dict[contig_key][0].split(':')[1][1:]) > overall_best_score:
                exonerate_best_hit_dict[contig_key] = 'overall_best_hit : %s' % overall_best_score
    else:
        exonerate_no_filtered_allele_no_asso_contig_bool_dict[contig_key] = False

### Managed up to here 19/05/2017

In [208]:
exonerate_no_filtered_allele_no_asso_contig_bool_dict

{'evm.model.hcontig_049_206.1': False,
 'evm.model.hcontig_049_206.5': False,
 'evm.model.hcontig_057_232.4': False,
 'evm.model.pcontig_225.5': True,
 'evm.model.pcontig_225.9': True}

In [209]:
exonerate_no_filtered_allele_gene_genome_hit_no_asso_contig_dict

{'evm.model.hcontig_049_206.1': ['pcontig_020_9602_70016 : False'],
 'evm.model.hcontig_049_206.5': ['pcontig_020_47632_110942 : False'],
 'evm.model.hcontig_057_232.4': ['pcontig_046_519632_580332 : False',
  'pcontig_187_1_31011 : False',
  'pcontig_019_1038363_1099034 : False',
  'pcontig_000_625602_686302 : False',
  'pcontig_042_393900_454599 : False',
  'pcontig_001_746191_806763 : False'],
 'evm.model.pcontig_225.5': ['hcontig_086_002_32952_93828 : True',
  'hcontig_002_028_1198296_1259172 : True',
  'hcontig_027_006_207890_268766 : True',
  'hcontig_012_028_204944_265820 : True'],
 'evm.model.pcontig_225.9': ['hcontig_086_002_41119_103356 : True']}

In [200]:
exonerate_no_filtered_allele_gene_genome_hit_asso_contig_dict

{'evm.model.hcontig_014_192.6': ['pcontig_014_1256595_1318018 : False'],
 'evm.model.pcontig_225.5': ['hcontig_225_001_1_50732 : False']}

In [201]:
exonerate_no_filtered_allele_asso_contig_bool_dict

{'evm.model.hcontig_014_192.6': False, 'evm.model.pcontig_225.5': False}

In [202]:
exonerate_best_hit_dict

{'evm.model.pcontig_225.5': ['dummy : 0']}

In [182]:
if contig_key in exonerate_best_hit_dict.keys():
    if 
    print('No')

In [185]:
print(fname.split('/')[-1].split('.')[0])

pcontig_014_1256595_1318018


In [155]:
no_filtered_allele_gene_genome_hit_no_asso_contig_dict

{'evm.model.pcontig_225.5': [SeqRecord(seq=Seq('ATTGGAGTGTCATTGAGGGGTCATTGGAGGGTCATTGGGGGAAGGTTGGTGGCA...gat', 'fasta'), id='hcontig_086_002_32952_93828', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
  SeqRecord(seq=Seq('GGGCTAAGGTTCGAAAGAATTTAAGAAAAGAAGGAGATGAAGAAGATGAAGAAG...GCT', 'fasta'), id='hcontig_002_028_1198296_1259172', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
  SeqRecord(seq=Seq('GCTGACAATGTACATTGATCTCCTGATCTAATCCCTGGCTTTCAAGAGTACTTT...TAC', 'fasta'), id='hcontig_012_028_204944_265820', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
  SeqRecord(seq=Seq('AGGTTTGGGTAGTGCGACTATGTGGTAAATGGTGATGATGGTAATGCATGACTC...CCT', 'fasta'), id='hcontig_027_006_207890_268766', name='<unknown name>', description='<unknown description>', dbxrefs=[])]}

In [140]:
test = gene_genome_hits_df.pivot_table(columns=['Protein_ID', 'Contig'], aggfunc={'start' : 'min', 'end':'min'}).unstack()

In [141]:
test

Contig                           hcontig_000_003  hcontig_000_005  \
      Protein_ID                                                    
start evm.model.pcontig_000.113         489890.0              NaN   
      evm.model.pcontig_000.114         490669.0              NaN   
      evm.model.pcontig_000.118              NaN              NaN   
      evm.model.pcontig_000.121              NaN              NaN   
      evm.model.pcontig_000.128         744498.0              NaN   
      evm.model.pcontig_000.130              NaN              NaN   
      evm.model.pcontig_000.147         666300.0              NaN   
      evm.model.pcontig_000.152         683263.0              NaN   
      evm.model.pcontig_000.185              NaN              NaN   
      evm.model.pcontig_000.186          21856.0              NaN   
      evm.model.pcontig_000.188         819170.0              NaN   
      evm.model.pcontig_000.2             1022.0              NaN   
      evm.model.pcontig_000.200         865171.0              NaN   
      evm.model.pcontig_000.208         475849.0              NaN   
      evm.model.pcontig_000.211         895314.0              NaN   
      evm.model.pcontig_000.225         981629.0              NaN   
      evm.model.pcontig_000.231         995623.0              NaN   
      evm.model.pcontig_000.236        1007881.0              NaN   
      evm.model.pcontig_000.237        1029345.0              NaN   
      evm.model.pcontig_000.24          134177.0              NaN   
      evm.model.pcontig_000.276        1158140.0              NaN   
      evm.model.pcontig_000.283        1190025.0              NaN   
      evm.model.pcontig_000.287          10316.0              NaN   
      evm.model.pcontig_000.29          157968.0              NaN   
      evm.model.pcontig_000.294        1251382.0              NaN   
      evm.model.pcontig_000.295         193615.0              NaN   
      evm.model.pcontig_000.304        1308194.0              NaN   
      evm.model.pcontig_000.323              NaN              NaN   
      evm.model.pcontig_000.33          168662.0              NaN   
      evm.model.pcontig_000.341        1464980.0              NaN   
...                                          ...              ...   
end   evm.model.pcontig_187.5           178919.0              NaN   
      evm.model.pcontig_188.1                NaN              NaN   
      evm.model.pcontig_189.3                NaN              NaN   
      evm.model.pcontig_189.5                NaN              NaN   
      evm.model.pcontig_189.6                NaN              NaN   
      evm.model.pcontig_191.1                NaN              NaN   
      evm.model.pcontig_195.5                NaN              NaN   
      evm.model.pcontig_200.7                NaN              NaN   
      evm.model.pcontig_203.1                NaN              NaN   
      evm.model.pcontig_203.12               NaN              NaN   
      evm.model.pcontig_203.5                NaN              NaN   
      evm.model.pcontig_204.1                NaN              NaN   
      evm.model.pcontig_204.6                NaN              NaN   
      evm.model.pcontig_205.2                NaN              NaN   
      evm.model.pcontig_207.5                NaN              NaN   
      evm.model.pcontig_207.6                NaN              NaN   
      evm.model.pcontig_218.1                NaN              NaN   
      evm.model.pcontig_218.6                NaN              NaN   
      evm.model.pcontig_221.1                NaN              NaN   
      evm.model.pcontig_221.4                NaN              NaN   
      evm.model.pcontig_223.2                NaN              NaN   
      evm.model.pcontig_223.4                NaN              NaN   
      evm.model.pcontig_223.8                NaN              NaN   
      evm.model.pcontig_225.5                NaN              NaN   
      evm.model.pcontig_225.9                NaN              NaN  

In [124]:
test

Contig,hcontig_001_021
end,770509
start,770465


In [113]:
tmp_protein_id_df

,Contig,start,end,blast_query,e-value,strand,Protein_ID,asso_contig
38756,hcontig_001_021,770628,770821,evm.TU.pcontig_001.325,7.000000e-66,+,evm.model.pcontig_001.325,True
38757,hcontig_001_021,770465,770509,evm.TU.pcontig_001.325,3.000000e-10,+,evm.model.pcontig_001.325,True


In [105]:
tmp_protein_id_df.groupby('Contig')['end'].max()

Contig
hcontig_001_021    770821
Name: end, dtype: int64

In [97]:
len(fa_protein_seq_dict)

30249

In [94]:
tmp_same_contig_df

,Contig,start,end,blast_query,e-value,strand,Protein_ID,asso_contig
1,hcontig_225_001,19857,20732,evm.TU.pcontig_225.5,0.000000e+00,+,evm.model.pcontig_225.5,True
7,hcontig_225_001,27287,28088,evm.TU.pcontig_225.9,0.000000e+00,+,evm.model.pcontig_225.9,True
13,hcontig_166_001,0,1367,evm.TU.pcontig_166.6,0.000000e+00,-,evm.model.pcontig_166.6,True
17,hcontig_189_001,9215,9760,evm.TU.pcontig_189.3,0.000000e+00,-,evm.model.pcontig_189.3,True
18,hcontig_189_001,14376,18616,evm.TU.pcontig_189.5,0.000000e+00,+,evm.model.pcontig_189.5,True
19,hcontig_189_001,16223,16879,evm.TU.pcontig_189.5,1.000000e-70,+,evm.model.pcontig_189.5,True
20,hcontig_189_001,16118,16774,evm.TU.pcontig_189.5,1.000000e-69,+,evm.model.pcontig_189.5,True
21,hcontig_189_001,16128,16694,evm.TU.pcontig_189.5,2.000000e-63,+,evm.model.pcontig_189.5,True
22,hcontig_189_001,16336,16904,evm.TU.pcontig_189.5,2.000000e-62,+,evm.model.pcontig_189.5,True
27,hcontig_189_001,20435,22664,evm.TU.pcontig_189.6,0.000000e+00,-,evm.model.pcontig_189.6,True


In [67]:
#all primary proteins no hit need to be split up into pwh and pwoh
p_contig_list = []
h_contig_list = []
for seq in SeqIO.parse(os.path.join(BASE_A_PATH, h_genome +'.fa'),'fasta'):
    h_contig_list.append(seq.id)
for seq in SeqIO.parse(os.path.join(BASE_A_PATH, p_genome +'.fa'), 'fasta'):
    p_contig_list.append(seq.id)
pwh_set = set([x[0:11].replace('h','p') for x in h_contig_list])
pwoh_set = set(p_contig_list) - pwh_set
print("P_contigs with h_contig are %i and without %i" % (len(pwh_set), len(pwoh_set)))

P_contigs with h_contig are 99 and without 57


In [58]:
np.intersectaaa1d(no_filtered_alleles, no_gene_genome_hits_df.Protein_ID.unique(), assume_unique= True)

array(['evm.model.pcontig_000.113', 'evm.model.pcontig_000.114',
       'evm.model.pcontig_000.118', ..., 'evm.model.pcontig_235.4',
       'evm.model.pcontig_248.2', 'evm.model.pcontig_248.3'], dtype=object)

In [59]:
np.setdiff1d(no_filtered_alleles, no_gene_genome_hits_df.Protein_ID.unique(), assume_unique= True)

array(['evm.model.pcontig_000.120', 'evm.model.pcontig_000.131',
       'evm.model.pcontig_000.19', ..., 'evm.model.pcontig_010.16',
       'evm.model.pcontig_004.437', 'evm.model.pcontig_074.68'], dtype=object)

In [60]:
np.setdiff1d( no_gene_genome_hits_df.Protein_ID.unique(),no_filtered_alleles, assume_unique= True)

array([], dtype=object)

In [62]:
no_filtered_allele_gene_no_genome_hit

array([], dtype=object)

In [ ]:
!cut -f4 Pst_E104_v1_p_ctg.p_genes.no_filtered_blast_hit.fa.outfmt6.bed | sort | uniq | wc -l

In [ ]:
!cut -f4 Pst_E104_v1_h_ctg.h_genes.no_filtered_blast_hit.fa.outfmt6.bed | sort | uniq | wc -l

In [ ]:
#here track what happens with the no_besties hit. Do they not have protein blast hits? How many of the no protein 
#blast hits have not gene blast hit?
#this needs to include some folder tracking of gene.no_besties.fa that hits nothing significant 
#no_bbb in - no_bbb out = no_hits at all
no_gene_hits = {}
no_besties_blast_nt_bed = [x for x in os.listdir() if x.endswith('no_besties.fa.outfmt6.bed')]
no_besties_blast_nt_bed.sort()
for no_bbb, protein_blast in zip(no_besties_blast_nt_bed,outfmt6):
    no_bbb_no_protein_blast_df =''
    no_bbb_df_header = ['Contig', 'start', 'end', 'blast_query', 'e-value', 'strand']
    no_bbb_df = pd.read_csv(folder_p+no_bbb, header=None, names=no_bbb_df_header,  sep='\t')
    protein_blast_df = pd.read_csv(folder_p+protein_blast, header=None, sep='\t')
    no_bbb_df['protein_id'] = no_bbb_df['blast_query'].str.replace('evm.TU', 'evm.model')
    #this below is most likely correct ignores the fact that some no_bbb genes might have hit nothing
    #at all on the gene level
    no_bbb_no_protein_blast_df = no_bbb_df[~no_bbb_df['protein_id'].isin(protein_blast_df[0])]
    #these are the no_besties that didn't hit anything at the gene level
    key =''
    key = no_bbb.split('.')[0]
    no_gene_hits[key] = set(no_bestie_dict[key]) - set(no_bbb_df['protein_id'].unique())
    pd.DataFrame(list(no_gene_hits[key])).to_csv(key + '.gene.no_genome_blast_hit.txt', sep='\t', header=None, index=None)
    blast_p_no_bestie =''
    blast_p_no_bestie = len(no_bbb_df[no_bbb_df['protein_id'].isin(protein_blast_df[0])]['blast_query'].unique())
    print('This %i out of %i no_besties of %s had a blast hit which was not RBH' % \
          (blast_p_no_bestie, len(no_bestie_dict[key]),no_bbb.split('.')[0]))
    print('This %i out of %i no_besties of %s have no blast hit gene vs. other haplome' % \
         (len(no_gene_hits[key]),len(no_bestie_dict[key]),no_bbb.split('.')[0]))
    print("No gene hits that have a protein hit", len(set(no_gene_hits[key])- set(no_hits[key])), key)
    groups = no_bbb_no_protein_blast_df.groupby(by='blast_query')
    #now filter the dataframe by the smallest e-value for each group == blast_hit
    df_filtered = groups.apply(lambda g: g[g['e-value'] == g['e-value'].min()])
    df_filtered = df_filtered.reset_index(drop=True)
    df_filtered.iloc[:,0:6].to_csv(folder_p+no_bbb[:-4]+'.filteredbesthits.bed', sep='\t', header=None, index=None)

In [ ]:
#all primary proteins no hit need to be split up into pwh and pwoh
p_contig_list = []
h_contig_list = []
for seq in SeqIO.parse('Pst_E104_v1_h_ctg.fa', 'fasta'):
    h_contig_list.append(seq.id)
for seq in SeqIO.parse('Pst_E104_v1_p_ctg.fa', 'fasta'):
    p_contig_list.append(seq.id)

In [ ]:
pwh_set = set([x[0:11].replace('h','p') for x in h_contig_list])
pwoh_set = set(p_contig_list) - pwh_set
print("P_contigs with h_contig are %i and without %i" % (len(pwh_set), len(pwoh_set)))

In [ ]:
def pwh_filter (x):
    p_contig = x.split('.')[2]
    if p_contig in pwh_set:
        return 1
    else:
        return 0

In [ ]:
fa_protein_dict['Pst_E104_v1_p_ctg_pwh']= [x for x in fa_protein_dict['Pst_E104_v1_p_ctg'] if x.split('.')[2] in pwh_set]
fa_protein_dict['Pst_E104_v1_p_ctg_pwoh']= [x for x in fa_protein_dict['Pst_E104_v1_p_ctg'] if x.split('.')[2] in pwoh_set]
print(len(fa_protein_dict['Pst_E104_v1_p_ctg_pwh']), len(fa_protein_dict['Pst_E104_v1_p_ctg_pwoh']), len (fa_protein_dict['Pst_E104_v1_p_ctg']))


In [ ]:
p_txt = [x for x in os.listdir(folder_p) if x.split('.')[0] == 'Pst_E104_v1_p_ctg' and x.endswith('.txt')\
        and not 'pwh' in x and not 'pwoh' in x]

In [ ]:
p_txt

In [ ]:
#filter and summarize the p results based on pwh and pwoh 
p_txt = [x for x in os.listdir(folder_p) if x.split('.')[0] == 'Pst_E104_v1_p_ctg' and x.endswith('.txt')\
        and not 'pwh' in x and not 'pwoh' in x and not x.endswith('besties.txt')]
for x in p_txt:
    #print(x)
    df_p = pd.read_csv(x, header=None, sep='\t')
    #df_p.head()
    df_p['pwh'] = df_p[0].apply(pwh_filter)
    df_p[df_p['pwh'] == 1].to_csv(x[:-4]+'pwh.txt', sep ='\t', header=None, index=None)
    df_p[df_p['pwh'] == 0].to_csv(x[:-4]+'pwoh.txt', sep ='\t', header=None, index=None)
    print ('For pwh:')
    print('For this condition %s %i proteins out of %i (%.2f) are affected for pwh'% \
          (x, sum(df_p['pwh']),len(fa_protein_dict['Pst_E104_v1_p_ctg_pwh']), \
           sum(df_p['pwh'])/len(fa_protein_dict['Pst_E104_v1_p_ctg_pwh'])*100 ))
    print ('For pwoh:')e
    print('For this condition %s %i proteins out of %i (%i) are affected for pwoh'% \
         (x, len(df_p['pwh']) - sum(df_p['pwh']),len(fa_protein_dict['Pst_E104_v1_p_ctg_pwoh']),\
        (len(df_p['pwh']) - sum(df_p['pwh']))/len(fa_protein_dict['Pst_E104_v1_p_ctg_pwoh'])*100 ))
     

from Bio import SeqIO
import os

os.chdir('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p')
len_pwh = 0
len_pwoh = 0
for seq in SeqIO.parse('Pst_E104_v1_p_ctg.fa', 'fasta'):
    if seq.id in pwh_set:
        len_pwh = len_pwh + len(seq.seq)
    if seq.id in pwoh_set:
        len_pwoh = len_pwoh + len(seq.seq)
print("Lenght of pwoh %i, lenght of pwo %i, total length p %i" %(len_pwh,len_pwoh,len_pwh+len_pwoh ))

In [ ]:
def same_contig_blast(x,y):
    '''Function that checks if the blast hit in columne y is on the same contig as the the query sequence in
    column y.
    '''
    q_contig = x.split('.')[2].split('_')[1]
    hit_contig = y.split('_')[1]
    if q_contig == hit_contig:
        return True
    else:
        return False

In [ ]:
import pysam
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
nfb_gene_blast_bed = [x for x in os.listdir(folder_p) if x.endswith('no_filtered_blast_hit.fa.outfmt6.bed')]
nfb_gene_blast_bed.sort()
protein_dict_nfb_bhits = {}
protein_dict_nfb = {} #get a list of all proteins of nfb that don't have a hit when blasted at the gene level too
#get the fasta genome files
tmp_genome_files = ['Pst_E104_v1_p_ctg.fa', 'Pst_E104_v1_h_ctg.fa']
protein_fa_files = [x for x in os.listdir(folder_p) if x.endswith('anno.RepaseTPSI_filtered.protein.fa')]
for bed_file in nfb_gene_blast_bed:
    print('This %s is the current bed file being processed' % (bed_file))
    nfb_gene_blast_bed_df =''
    nfb_gene_blast_bed_df = pd.read_csv(bed_file, header=None, sep='\t' )
    #now add another column to the dateframe that stats if the hit and query are on the same contig
    nfb_gene_blast_bed_df['Same_contig'] = nfb_gene_blast_bed_df[3].combine(nfb_gene_blast_bed_df[0], func=same_contig_blast)
    #initialize some temporary df for filtering
    tmp_same_contig_df =''
    tmp_best_hits_df =''
    tmp_groups =''
    tmp_best_hits_filtered =''
    #get all blast hits that are on the same contig
    tmp_same_contig_df = nfb_gene_blast_bed_df[nfb_gene_blast_bed_df['Same_contig'] == True]
    #get the best remaining blast hit(s)
    tmp_best_hits_df = nfb_gene_blast_bed_df[nfb_gene_blast_bed_df['Same_contig'] == False].sort_values(by=[3,4])
    tmp_groups = tmp_best_hits_df.groupby(by=3)
    #now filter the dataframe by the smallest e-value for each group == Query/3
    tmp_best_hits_df_filtered = tmp_groups.apply(lambda g: g[g[4] == g[4].min()]) 
    tmp_best_hits_df_filtered = tmp_best_hits_df_filtered.reset_index(drop=True)
    nfb_gene_blast_bed_df_filtered = ''
    nfb_gene_blast_bed_df_filtered = pd.concat([tmp_best_hits_df_filtered, tmp_same_contig_df]).sort_values(by=[3,4]).reset_index(drop=True)
    #now get the loop through the df. pull out the protein sequences and corresponding hits. save them to new folders.
    #extend the script. 
    #get all the blasted sequences that had a hit == unique querries
    tmp_queries = ''
    tmp_queries = nfb_gene_blast_bed_df_filtered[3].unique()
    #get all the protein sequences in a dictionary with protein ID being the key and the values being a SeqIO object
    #get the fasta genome files
    tmp_genome_file = [x for x in tmp_genome_files if not x.startswith(bed_file.split('.')[0])][0]
    genome_fa = ''
    genome_fa = pysam.FastaFile(tmp_genome_file)
    tmp_queries_id = [x.replace('TU', 'model') for x in tmp_queries]
    tmp_protein_fa_file = [x for x in protein_fa_files if x.startswith(bed_file.split('.')[0])][0]
    for seq in SeqIO.parse(open(tmp_protein_fa_file), 'fasta'):
            if seq.id in tmp_queries_id:
                protein_dict_nfb[seq.id] = seq
    #add this tmp_protein_dict_nfb to the full protein dict to keep track
    #check why only one file gets processed.
    
    
    #make a dict that gets the blast hit sequences in +30kb each side for alignments of protein sequences on top of them. 
    #The value of this dict will be a list of SeqIO objects
    
    tmp_list = [] #tmp_list to save the SeqIO objects for the blast hits in
    print(len(tmp_queries))
    for query in tmp_queries:
        #print(query)
        tmp_list = []
        tmp_df = nfb_gene_blast_bed_df_filtered[nfb_gene_blast_bed_df_filtered[3] == query]
        #do groupby instead here on columns one. Take min of column 1 and max of column 2 as start/stop +-
        #this avoids to mess around with mutliple hits on the same contig
        tmp_hit = tmp_df[0].unique()
        for hit in tmp_hit:
            tmp_df_2 = tmp_df[tmp_df[0] == hit]
            start = tmp_df_2[1].min() - 30000
            if start < 1:
                start = 1
            end = tmp_df_2[2].max() + 30000
            seq = genome_fa.fetch(hit, start, end)
            seq_r = '' #initialize empty SeqIO record
            seq_id = hit + '_' + str(start) + '_' + str(end)
            seq_ob = Seq(seq)
            seq_ob.alphabet = 'fasta'
            seq_r = SeqRecord(seq_ob)
            seq_r.id = seq_id
            tmp_list.append(seq_r)
        protein_dict_nfb_bhits[query] = tmp_list

In [ ]:
#generate a dicts for the summary dataframes for no_filtered_blast_hits
no_filtered_blast_sdf = {}
keys = no_filtered_blast_dict.keys()
for key in keys:
    tmp_df = ''
    tmp_df = pd.DataFrame.from_dict(no_filtered_blast_dict[key])
    tmp_df.rename(columns={0:'gene_model'}, inplace = True)
    no_filtered_blast_sdf[key] = tmp_df

#now add the length column to the data frame by using the fa_protein_length_dict 
for key in keys:
    tmp_length_df = ''
    tmp_length_df = pd.DataFrame.from_dict(fa_protein_length_dict[key], orient='index')
    tmp_length_df['gene_model'] = tmp_length_df.index
    tmp_length_df.reset_index(inplace=True, drop=True)
    tmp_length_df.rename(columns={0:'protein_length'}, inplace=True)
    tmp_length_df = tmp_length_df[tmp_length_df['gene_model'].isin(no_filtered_blast_sdf[key]['gene_model'])]
    tmp_length_df.reset_index(inplace=True, drop = True)
    no_filtered_blast_sdf[key] = pd.merge(tmp_length_df, no_filtered_blast_sdf[key])

#now add colum to the dataframe if or no the protein had a gene blast hit
for key in keys:
    tmp_gene_hit_df = ''
    #get the right df from no_filtered_gene_blast_hits
    tmp_gene_hit_df = nfb_gene_blast_bed_df_filtered_dict[key]
    #make a true false series if gene_models are having a blast hit
    tmp_gene_hit_bol_series = no_filtered_blast_sdf[key]['gene_model'].isin(tmp_gene_hit_df[3].str.replace('TU', 'model'))
    no_filtered_blast_sdf[key]['gene_hit'] = tmp_gene_hit_bol_series

In [ ]:
os.chdir(working_dir)

In [ ]:
#now loop over the dicts protein_dict_nfb_bhits and protein_dict_nfb with the keys and print out the sequences in a 
#new folder for each hit and write a script for this later
#exonerate folder initially was based on each blast hit form blast previously using interrow over the tmp_df above. 
#in some cases this lead to hundreds of hits on the same contig often in close proximity. This has been reduced 
#to one selected contig sequence per hit.

In [ ]:
#make new folder for exonerate
working_dir = os.path.abspath(folder_p)
exonerate_folder = os.path.join(working_dir, 'exonerate_2')
if not os.path.exists(exonerate_folder):
    os.mkdir(exonerate_folder)
protein_keys = [x.replace('TU', 'model') for x in protein_dict_nfb_bhits.keys()]
for key in protein_keys:
    new_folder = os.path.join(exonerate_folder, key)
    if not os.path.exists(new_folder):
        os.mkdir(new_folder)
    os.chdir(new_folder)
    out_p_f = open(key+'.fa', 'w')
    SeqIO.write(protein_dict_nfb[key], out_p_f, 'fasta')
    out_p_f.close()
    p_key = key.replace('model', 'TU')
    for seq in protein_dict_nfb_bhits[p_key]:
        out_t_f = open(seq.id + '.fa', 'w')
        SeqIO.write(seq, out_t_f, 'fasta')
        out_t_f.close()
    os.chdir(working_dir)
    

In [ ]:
#make write exonerate script
working_dir = os.path.abspath(folder_p)
os.chdir(working_dir)
protein_keys = [x.replace('TU', 'model') for x in protein_dict_nfb_bhits.keys()]
out_exonerate = open('exonerate_alignments_vulgar2.sh', 'w')
out_exonerate.write('#!/bin/bash\n')
for key in protein_keys:
    new_folder = os.path.join(exonerate_folder, key)
    protein_file_name = key+'.fa'
    p_key = key.replace('model', 'TU')
    out_exonerate.write('cd %s\n'% (new_folder))
    for seq in protein_dict_nfb_bhits[p_key]:
        target_file_name = seq.id + '.fa'
        out_exonerate.write('exonerate --model protein2genome --percent 20 -q %s -t %s --showalignment False -S > %s.vulgar_exn\n'\
                           %(protein_file_name, target_file_name,target_file_name ))
out_exonerate.write('cd %s\n' %(working_dir))
out_exonerate.close()
os.chdir(working_dir)

In [ ]:
#! bash exonerate_alignments_vulgar2.sh

In [ ]:
#make write exonerate script
working_dir = os.path.abspath(folder_p)
os.chdir(working_dir)
protein_keys = [x.replace('TU', 'model') for x in protein_dict_nfb_bhits.keys()]
out_exonerate = open('exonerate_alignments2.sh', 'w')
out_exonerate.write('#!/bin/bash\n')
for key in protein_keys:
    new_folder = os.path.join(exonerate_folder, key)
    protein_file_name = key+'.fa'
    p_key = key.replace('model', 'TU')
    out_exonerate.write('cd %s\n'% (new_folder))
    for seq in protein_dict_nfb_bhits[p_key]:
        target_file_name = seq.id + '.fa'
        out_exonerate.write('exonerate --model protein2genome --percent 20 -q %s -t %s --showalignment -S > %s.exn\n'\
                           %(protein_file_name, target_file_name,target_file_name ))
out_exonerate.write('cd %s\n' %(working_dir))
out_exonerate.close()
os.chdir(working_dir)

In [ ]:
#!bash exonerate_alignments2.sh

In [ ]:
#no loop through the exonerate vulgar result and generate a dictionray of the results
#if hsps query range == (0, query_length) and not F in .vulgar_comp it is likely that the alignment is actually good
#and and the gene model might have been dropped for another reason
working_dir = os.path.abspath(folder_p)
exonerate_folder = os.path.join(working_dir, 'exonerate_2')
protein_keys = [x.replace('TU', 'model') for x in protein_dict_nfb_bhits.keys()]
#a dict that has the gene model as key and the results of exonerate (True/False) as value.
exonerate_dict = {}
exonerate_best_hit_dict = {}
#now loop through the exonerate folders
for key in protein_keys:
    new_folder = os.path.join(exonerate_folder, key)
    os.chdir(new_folder)
    if key.split('.')[2].startswith('h'):
        query_length = fa_protein_length_dict['Pst_E104_v1_h_ctg'][key]
    else:
        query_length = fa_protein_length_dict['Pst_E104_v1_p_ctg'][key]
    counter = 0
    vulgar_exn = [x for x in os.listdir() if x.endswith('vulgar_exn')]
    opt_query_range = (0, query_length)
    #loop through vulgar parser and see if hit is 
    best_score = 0
    best_hit = ''
    for fname in vulgar_exn:
        result = SearchIO.parse(fname, 'exonerate-vulgar') 
        for hit in result:
            #loop through all hsps hits
            for hsps in hit.hsps:
                hsps_range = hsps.query_range
                vulgar_list = hsps.vulgar_comp.strip(' ').split(' ')
                #print(hsps_range, vulgar_list)
                #this is the contition for something being a potential protein alignment that
                #has been left out
                if hsps_range == opt_query_range and 'F' not in vulgar_list:
                    counter += 1
                    if hsps.score > best_score:
                        best_hit = hsps.hit_id
                    #print(key)
    if counter > 0:
        exonerate_dict[key] = True
        exonerate_best_hit_dict[key] = best_hit
    else:
        exonerate_dict[key] = False
    
    os.chdir(working_dir)

In [ ]:
#now add colum to the dataframe if or not a protein sequence could be aligned back to the gene blast hit using exonerate
#plus a column that describes the best exonerate hit
keys = no_filtered_blast_dict.keys()
for key in keys:
    #pull in the two exonerate df to be combined with the no_filtered_blast_sdf dataframe
    tmp_exonerate_dict_df = pd.DataFrame.from_dict(exonerate_dict, orient='index')
    tmp_exonerate_dict_df['gene_model'] = tmp_exonerate_dict_df.index
    tmp_exonerate_dict_df.rename(columns={0:'exonerate_hit'}, inplace=True)
    tmp_exonerate_dict_df.reset_index(inplace=True, drop=True)
    tmp_exonerate_best_hit_dict_df = pd.DataFrame.from_dict(exonerate_best_hit_dict, orient='index')
    tmp_exonerate_best_hit_dict_df['gene_model'] = tmp_exonerate_best_hit_dict_df.index
    tmp_exonerate_best_hit_dict_df.rename(columns={0:'exonerate_best_hit'}, inplace=True)
    tmp_exonerate_best_hit_dict_df.reset_index(inplace=True, drop=True)
    tmp_exonerate_df = pd.merge(tmp_exonerate_best_hit_dict_df,tmp_exonerate_dict_df)  #same length so no out neccessary
    tmp_exonerate_df.reset_index(inplace=True, drop=True)
    if 'h_ctg' in key:
        tmp_exonerate_df = tmp_exonerate_df[tmp_exonerate_df.gene_model.str.contains('hcontig')]
    else:
        tmp_exonerate_df =tmp_exonerate_df[tmp_exonerate_df.gene_model.str.contains('pcontig')]
    tmp_exonerate_df.reset_index(inplace=True, drop=True)
    no_filtered_blast_sdf[key]= pd.merge( no_filtered_blast_sdf[key],tmp_exonerate_df, how='outer').fillna(value=False)

In [ ]:
#add another column for genes being on pwh or not
for key in keys:
    df_p['pwh'] = df_p[0].apply(pwh_filter)
    no_filtered_blast_sdf[key]['pwh'] = no_filtered_blast_sdf[key]['gene_model'].apply(pwh_filter)

In [ ]:
#now save the dataframes for now
for key in keys:
    filename = key + '.no_filtered_blast_sdf.tab'
    no_filtered_blast_sdf[key].to_csv(filename, index=None, sep='\t')

In [ ]:
!pwd

In [ ]:
#next step is to get the gffs for all the the no_filtered blast hits for both p and h contigs that do not have a exonerate-hit
#in case of the p contigs these need to be compared to the unqie bed dataframes using pybed tools

#pull in gff dataframe and parse out gene model into a new column use this column for filtering down the dataframe
#write this out again and load as bedfile







In [ ]:
def col_8_id(x):
    import re
    pattern = r'ID=([a-zA-Z0-9_.]*);'
    regex = re.compile(pattern)  
    m = regex.search(x)
    match = m.groups()[0].replace('TU', 'model')
    if match.startswith('cds.'):
        match = match[4:]
    if 'exon' in match:
        _list = match.split('.')
        match = '.'.join(_list[:-1])
    return match


In [ ]:
#read in gene annotation gff files for downstream analysis
annotation_folder = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/122016_assembly'
gene_anno_gffs = [x for x in os.listdir(annotation_folder) if x.endswith('anno.RepaseTPSI_filtered.gff3') and x.startswith('Pst_E104_v11')]
gene_anno_gff_dict = {}
for file in gene_anno_gffs:
    full_file_path = os.path.join(annotation_folder, file)
    tmp_df =  pd.read_csv(full_file_path, header=None, sep='\t')
    tmp_df['gene_model'] = tmp_df[8].apply(col_8_id)
    gene_anno_gff_dict[file.split('.')[0].replace('11', '1')] =  tmp_df

In [ ]:
#filter gene annotation gff files for downstream analysis and save to file
os.chdir(working_dir)
keys = no_filtered_blast_sdf.keys()
no_filtered_blast_gffs_plus = {}
for key in keys:
    tmp_df_gff = ''
    tm_df_nfb = ''
    tmp_df = ''
    tmp_df_gff = gene_anno_gff_dict[key]
    tmp_df_nfb = no_filtered_blast_sdf[key]
    tmp_df = tmp_df_gff[tmp_df_gff['gene_model'].isin(tmp_df_nfb['gene_model'].tolist())]
    tmp_df.reset_index(inplace=True, drop=True)
    no_filtered_blast_gffs_plus[key] = tmp_df
    file_name = key+'.anno.no_filtered_blast.gff3'
    no_filtered_blast_gffs_plus[key].iloc[:,range(0,9)].to_csv(file_name, header=None, sep='\t', index=None)
    gene_file_name = key+'.gene.no_filtered_blast.gff3'
    no_filtered_blast_gffs_plus[key][no_filtered_blast_gffs_plus[key][2] == 'gene'].iloc[:,range(0,9)].to_csv(gene_file_name, header=None, sep='\t', index=None)
    tmp_df_gff = ''
    tm_df_nfb = ''
    tmp_df = ''

In [ ]:
#now lead into the  Pst_E104_v1_p_ctg.gene.no_filtered_blast.gff3 and the Pst_E104_v1_ph_ctg.ph_p_homo_cov.bed as 
#bed files and do an intersect
from pybedtools import BedTool
cov_folder = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/COV'
homo_cov_ph_p = 'Pst_E104_v1_ph_ctg.ph_p_homo_cov.bed'
homo_cov_ph_p_bed = BedTool(os.path.join(cov_folder, homo_cov_ph_p))

In [ ]:
#read in no blast hit gene gff of primary contigs 
no_filtered_blast_gene_p = 'Pst_E104_v1_p_ctg.gene.no_filtered_blast.gff3'
no_filtered_blast_gene_p_bed = BedTool(os.path.join(working_dir, no_filtered_blast_gene_p))

In [ ]:
#get the id of all genes of no filtered blast hits that overlap at all with homo coverage of ph mapping on p
gene_ids_ph_p_homo = []
for x in no_filtered_blast_gene_p_bed.intersect(homo_cov_ph_p_bed):
    y = col_8_id(x[8])
    gene_ids_ph_p_homo.append(y)

In [ ]:
#now filter the no_filtered_blast_gffs_plus_p and save the data frame as gff file again
no_filtered_blast_gffs_plus_p = no_filtered_blast_gffs_plus['Pst_E104_v1_p_ctg']
no_filtered_blast_gffs_plus_p[~no_filtered_blast_gffs_plus_p['gene_model'].isin(gene_ids_ph_p_homo)]
no_filtered_blast_gffs_plus_p_no_homo = no_filtered_blast_gffs_plus_p[~no_filtered_blast_gffs_plus_p['gene_model'].isin(gene_ids_ph_p_homo)]
no_filtered_blast_gffs_plus_p_no_homo.iloc[:,range(0,9)].to_csv\
('Pst_E104_v1_p_ctg.anno.no_filtered_blast.no_homo.gff3', header=None, sep='\t', index=None)

In [ ]:
#add new column to the summary dataframe that looks for ph_h_homo coverage (this is actually only true for contigs with 
# mean coverage < 2000)
tmp_nfb_sdf_p = no_filtered_blast_sdf['Pst_E104_v1_p_ctg']
tmp_homo_p_series = tmp_nfb_sdf_p['gene_model'].isin(gene_ids_ph_p_homo)
tmp_nfb_sdf_p['ph_p_homo_cov'] = tmp_homo_p_series
no_filtered_blast_sdf['Pst_E104_v1_p_ctg'] = tmp_nfb_sdf_p
filename = 'Pst_E104_v1_p_ctg' + '.no_filtered_blast_sdf.tab'
no_filtered_blast_sdf['Pst_E104_v1_p_ctg'].to_csv(filename, index=None, sep='\t')

## up to here looks good consider that high cov contigs are not filtered out yet

In [ ]:
tmp_nfb_sdf_p[(tmp_nfb_sdf_p['pwh'] == False)&(tmp_nfb_sdf_p['exonerate_hit'] == True)&(tmp_nfb_sdf_p['ph_p_homo_cov'] == True)]

In [ ]:
no_filtered_blast_sdf['Pst_E104_v1_p_ctg'][no_filtered_blast_sdf['Pst_E104_v1_p_ctg'].pwh == 0].sum()

In [ ]:
no_filtered_blast_sdf['Pst_E104_v1_p_ctg'].head()

In [ ]:
keys

In [ ]:
no_filtered_blast_sdf['Pst_E104_v1_h_ctg'][no_filtered_blast_sdf['Pst_E104_v1_h_ctg']['exonerate_hit'] == False]['protein_length'].mean()

In [ ]:
no_filtered_blast_sdf['Pst_E104_v1_h_ctg'][no_filtered_blast_sdf['Pst_E104_v1_h_ctg'].pwh == 0]['protein_length'].mean()

In [ ]:
no_filtered_blast_sdf['Pst_E104_v1_h_ctg']['protein_length'].hist(bins=20,alpha=0.5, color='g')
no_filtered_blast_sdf['Pst_E104_v1_p_ctg']['protein_length'].hist(bins=20,alpha=0.5, color='r')

In [ ]:
len(no_filtered_blast_sdf['Pst_E104_v1_h_ctg'][no_filtered_blast_sdf['Pst_E104_v1_h_ctg']['exonerate_hit'] == False])/15000

In [ ]:
len(nfb_gene_blast_bed_df_filtered[3].unique())